<a href="https://colab.research.google.com/github/DolevSeren/DI-Bootcamp/blob/main/W7D3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
sentence = "BERT is a powerful model for natural language understanding."


In [4]:
tokens = tokenizer.tokenize(sentence)
print(tokens)


['bert', 'is', 'a', 'powerful', 'model', 'for', 'natural', 'language', 'understanding', '.']


In [5]:
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
print(inputs)


{'input_ids': tensor([[  101, 14324,  2003,  1037,  3928,  2944,  2005,  3019,  2653,  4824,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [6]:
input_ids = inputs['input_ids'][0]
tokens_with_special = tokenizer.convert_ids_to_tokens(input_ids)
print(tokens_with_special)


['[CLS]', 'bert', 'is', 'a', 'powerful', 'model', 'for', 'natural', 'language', 'understanding', '.', '[SEP]']


In [7]:
from transformers import pipeline

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [8]:
result = sentiment_pipeline("I absolutely loved the movie! It was fantastic.")
print(result)


[{'label': 'POSITIVE', 'score': 0.9998797178268433}]


In [9]:
print(sentiment_pipeline("This product is terrible. I'm so disappointed."))
print(sentiment_pipeline("Not bad, but could be better."))
print(sentiment_pipeline("The service was okay, nothing special."))


[{'label': 'NEGATIVE', 'score': 0.9997947812080383}]
[{'label': 'NEGATIVE', 'score': 0.8015893697738647}]
[{'label': 'NEGATIVE', 'score': 0.9862403273582458}]


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

class BERTSentimentAnalyzer:
    def __init__(self, model_name="distilbert-base-uncased-finetuned-sst-2-english"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)

    def predict(self, text):
        # שלב 1: טוקניזציה והכנת טנסורים
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)

        # שלב 2: הפעלת המודל
        with torch.no_grad():
            outputs = self.model(**inputs)

        # שלב 3: המרת logits לסיכוי (softmax)
        probs = F.softmax(outputs.logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
        confidence = probs[0][predicted_class].item()

        label = self.model.config.id2label[predicted_class]
        return {"label": label, "confidence": round(confidence, 4)}


In [11]:
analyzer = BERTSentimentAnalyzer()

print(analyzer.predict("This app is amazing, I use it every day!"))
print(analyzer.predict("I hate everything about this product."))
print(analyzer.predict("It's okay, but I expected more."))


{'label': 'POSITIVE', 'confidence': 0.9999}
{'label': 'NEGATIVE', 'confidence': 0.9995}
{'label': 'POSITIVE', 'confidence': 0.9159}


In [12]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

class BERTNamedEntityRecognizer:
    def __init__(self, model_name="dslim/bert-base-NER"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(model_name)
        self.label_map = self.model.config.id2label

    def recognize_entities(self, text):
        # Tokenize input
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, is_split_into_words=False)
        with torch.no_grad():
            outputs = self.model(**inputs)

        # Get logits and predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2)[0]

        tokens = self.tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        entities = []

        for token, pred in zip(tokens, predictions):
            label = self.label_map[pred.item()]
            if label != "O":
                entities.append((token, label))

        return entities


In [13]:
ner = BERTNamedEntityRecognizer()

sample_text = "Barack Obama was born in Hawaii and worked at the White House in Washington."
entities = ner.recognize_entities(sample_text)

for token, label in entities:
    print(f"{token:15} → {label}")


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Barack          → B-PER
Obama           → I-PER
Hawaii          → B-LOC
White           → B-LOC
House           → I-LOC
Washington      → B-LOC


Exercise 5 Summary – Comparing BERT and GPT
In this exercise, we explored the key differences between BERT and GPT, two foundational models in natural language processing. BERT is based on a Transformer encoder-only architecture and is designed primarily for understanding text, using techniques like Masked Language Modeling and Next Sentence Prediction. In contrast, GPT relies on a decoder-only architecture and excels at generating text, predicting the next word in a sequence. BERT performs better in tasks such as sentiment analysis, question answering, and named entity recognition, while GPT is more suited for text completion, summarization, and conversational AI. Ultimately, the choice between BERT and GPT depends on whether the task requires deep understanding or creative language generation.

RAG combines retrieval (powered by BERT or similar models) with generation (powered by GPT/BART).
BERT’s role is crucial for finding relevant, contextually-matching documents, which improves the accuracy and factual grounding of the generated response.